<a href="https://colab.research.google.com/github/MostafaMamdouh21/AI-Weather-Assistant/blob/main/Weather_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies and Imports

In [8]:
#Install dependencies
!pip install langchain-community -q
!pip install sentence-transformers -q
!pip install langchain faiss-cpu pypdf -q
!pip install bitsandbytes -q
!pip install gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [9]:
# Imports
import os
import torch
import requests
import re
import json
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS

## Install LLM

In [39]:
from huggingface_hub import login
import getpass

# Prompt the user to enter their token
token = getpass.getpass("Enter your Hugging Face token: ")
login(token=token)


Enter your Hugging Face token: ··········


In [11]:
pipe = pipeline(
    "text-generation",
    model="NousResearch/DeepHermes-3-Llama-3-8B-Preview",
    torch_dtype="auto",     # Enables float16 on GPU
    device_map="auto"       # Automatically uses GPU
)


config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Device set to use cuda:0


## Vector Database

In [12]:
# Load and split both PDFs
pdf_paths = ["/content/Weather Activity Clothing Database - A Comprehensive Global Guide.pdf", "/content/Weather Activity Clothing Database.pdf"]
documents = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    documents.extend(docs)

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(documents)

In [13]:
#Load Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-13-38819264.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# Create a FAISS vector database
vector_db = FAISS.from_documents(chunks, embedding_model)

# Save the built vector database locally
vector_db.save_local("vector_db")

In [15]:
# Load the vector DB
vector_db = FAISS.load_local("vector_db", embedding_model, allow_dangerous_deserialization=True)

## Assistant Functions

In [16]:
def extract_location_with_llm(user_input):
    prompt = f"""Extract the location mentioned in this sentence:
"{user_input}"

Only reply with the city or place name, nothing else.
"""

    response = pipe(prompt, max_new_tokens=20)[0]['generated_text']

    # Strip out the prompt from the output
    location = response.replace(prompt, "").strip().split('\n')[0]

    # Optional: Remove anything that's clearly not a location
    location = re.sub(r'[^\w\s]', '', location)  # remove punctuation
    location = location.strip()

    return location if location else None


In [17]:
def geocode_location(location_name):
    # Construct the Open-Meteo geocoding API URL with the provided location name
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location_name}&count=1"

    try:
        # Send a GET request to the geocoding API
        response = requests.get(geo_url)

        # Raise an exception if the response status is not successful (e.g., 404, 500)
        response.raise_for_status()

        # Parse the response as JSON
        data = response.json()

        # Check if the API returned any results
        if data.get("results"):
            # Get the first result (most relevant)
            location = data["results"][0]

            # Return the latitude and longitude of the location
            return location["latitude"], location["longitude"]
        else:
            # No matching location found
            print("Location not found.")
            return None, None

    except requests.exceptions.RequestException as e:
        # Handle any errors that occur during the API request
        print("Error in geocoding:", e)
        return None, None


In [18]:
def interpret_weather_code(code):
    # Define a dictionary that maps weather condition codes to human-readable descriptions and emojis
    code_map = {
        0: "Clear sky ☀️",
        1: "Mainly clear 🌤️",
        2: "Partly cloudy ⛅",
        3: "Overcast ☁️",
        45: "Fog 🌫️",
        48: "Depositing rime fog 🌫️",
        51: "Light drizzle 🌦️",
        53: "Moderate drizzle 🌦️",
        55: "Dense drizzle 🌧️",
        61: "Slight rain 🌧️",
        63: "Moderate rain 🌧️",
        65: "Heavy rain 🌧️",
        66: "Freezing rain ❄️🌧️",
        67: "Heavy freezing rain ❄️🌧️",
        71: "Slight snow ❄️",
        73: "Moderate snow ❄️",
        75: "Heavy snow ❄️",
        80: "Slight rain showers 🌦️",
        81: "Moderate rain showers 🌦️",
        82: "Violent rain showers 🌧️",
        95: "Thunderstorm ⛈️",
        96: "Thunderstorm with hail ⛈️",
        99: "Severe thunderstorm with hail ⛈️"
    }

    # If the code is not found, return "Unknown"
    return code_map.get(code, "Unknown")

In [19]:
def get_weather(latitude, longitude, temperature_unit='celsius'):
    # Determine the temperature unit to send to the API (either 'celsius' or 'fahrenheit')
    temp_unit_api = 'fahrenheit' if temperature_unit.lower() == 'fahrenheit' else 'celsius'

    # Construct the URL for the Open-Meteo API with the given latitude, longitude, and temperature unit
    weather_url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current_weather=true&temperature_unit={temp_unit_api}"
    )

    try:
        # Send a GET request to the weather API
        response = requests.get(weather_url)

        # Raise an exception if the response status indicates an error (e.g., 404, 500)
        response.raise_for_status()

        # Parse the response JSON into a Python dictionary
        data = response.json()

        # Extract the current weather data (if available)
        current = data.get("current_weather", {})

        # Get the weather condition code from the current weather data
        weather_code = current.get("weathercode")

        # Return a dictionary containing temperature, wind speed, weather description, and unit
        return {
            "temperature": current.get("temperature"),
            "wind_speed": current.get("windspeed"),
            "weather_description": interpret_weather_code(weather_code),  # Convert code to readable description
            "unit": temp_unit_api
        }

    except requests.exceptions.RequestException as e:
        # Print an error message if the API request fails
        print("Error fetching weather data:", e)
        return None

In [20]:
def generate_semantic_query(user_query, weather_data):
    """Generate a semantic search query using the LLM."""

    # Construct a description string of the current weather using the provided weather data
    weather_description = (
        f"Temperature: {weather_data['temperature']}°{weather_data['unit'].capitalize()}, "
        f"Wind Speed: {weather_data['wind_speed']} km/h, "
        f"Weather Description: {weather_data['weather_description']}"
    )

    # Create a prompt that combines the user query and current weather conditions to generate a relevant search query for a vector database
    prompt = f"""You are an intelligent assistant. A user asked the following question:

"{user_query}"

The current weather conditions are:
{weather_description}

Generate a concise and coherent semantic search query that could be used to look up relevant information in a vector database (e.g., about clothing, outdoor activities, or travel tips). Only return the search query, nothing else.
"""

    # Send the prompt to the language model pipeline and limit the output to 50 new tokens
    response = pipe(prompt, max_new_tokens=50)[0]["generated_text"]

    # Remove the original prompt from the generated text
    generated_query = response.replace(prompt, "").strip().split('\n')[0]

    # Return the cleaned semantic search query
    return generated_query

In [21]:
def get_similarity_results(vector_db, semantic_query, k=3):
    """
    Perform a similarity search on the vector database and return the top k results.

    Parameters:
    - vector_db: The vector database object with a similarity_search method.
    - semantic_query: The query string for semantic similarity search.
    - k: Number of top results to retrieve.

    Returns:
    - List of retrieved documents.
    """
    return vector_db.similarity_search(semantic_query, k=k)


In [22]:
def ask_llm_from_results(pipe, query, weather_data, results, max_tokens=512, temperature=0.7):
    """
    Uses a Hugging Face chat-style LLM to generate a comprehensive answer to a user's query,
    considering current weather data and relevant documents.

    Parameters:
        pipe: Hugging Face pipeline object for text generation.
        query (str): The user's question.
        weather_data (dict): Dictionary containing weather details (temperature, wind_speed, unit, and description).
        results (List[Document]): List of LangChain Document objects.
        max_tokens (int): Max tokens to generate. Default is 512.
        temperature (float): Sampling temperature. Default is 0.7.

    Returns:
        str: The assistant's generated response.
    """

    # Format document context
    sources_context = "\n\n".join([f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(results)])

    # Format weather context
    weather_context = (
        f"Current Weather Conditions:\n"
        f"- Temperature: {weather_data['temperature']}°{weather_data['unit'].capitalize()}\n"
        f"- Wind Speed: {weather_data['wind_speed']} km/h\n"
        f"- Description: {weather_data['weather_description']}\n"
    )

    # Compose full prompt
    prompt = f"""You are a smart AI assistant that helps users based on the current weather conditions and available information sources.

Your goal is to:
- Explain the current weather clearly.
- Suggest suitable outdoor or indoor activities.
- Recommend clothing based on the weather.
- Use the provided sources for relevant tips or safety advice.

User's Question: {query}

{weather_context}

Information Sources:
{sources_context}

Now provide a helpful and concise response that addresses all the above points.

Answer:"""

    # Chat-style input
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant providing personalized weather-based advice using documents and weather data."},
        {"role": "user", "content": prompt}
    ]

    # Call the model
    response = pipe(messages, max_new_tokens=max_tokens, temperature=temperature)

    # Handle output format
    generated = response[0].get("generated_text")

    if isinstance(generated, str):
        # Standard string output — strip prompt and return answer
        answer = generated.replace(prompt, "").strip()
        return answer if answer else "⚠️ Assistant response not found."
    elif isinstance(generated, list):
        # Chat-style output — find assistant response
        for turn in generated:
            if turn.get("role") == "assistant":
                return turn.get("content", "").strip()
        return "⚠️ Assistant reply not found in structured output."
    else:
        return "⚠️ Unexpected response format."


In [23]:
def weather_assistant_verbose(user_input):
    # Step 1: Extract location name from user input
    location_name = extract_location_with_llm(user_input)
    if location_name == "Unknown":
        return "❌ Could not extract location.", "", ""

    # Step 2: Geocode location to get latitude and longitude
    latitude, longitude = geocode_location(location_name)
    if latitude is None or longitude is None:
        return f"❌ Could not geocode location: {location_name}", "", ""

    # Step 3: Fetch weather using lat/lon
    weather = get_weather(latitude, longitude)
    if not weather:
        return f"❌ Could not fetch weather for {location_name}.", "", ""

    # Format weather as string for display
    weather_str = f"{weather['temperature']}°C"

    # Step 4: Generate semantic query
    semantic_query = generate_semantic_query(user_input, weather)

    # Step 5: Vector search
    results = get_similarity_results(vector_db, semantic_query , k=3)

    # Step 6: Final LLM answer
    answer = ask_llm_from_results(pipe, user_input, weather, results)

    return location_name, weather_str, answer

## Functions Test

In [24]:
user_input = "What should I wear today in Egypt?"
location = extract_location_with_llm(user_input)

print(f"📍 Extracted Location: {location}")

📍 Extracted Location: Egypt


In [25]:
lat, lon = geocode_location(location)
if lat and lon:
  weather = get_weather(lat, lon, 'celsius')
  if weather:
    print(f"\n🌤️ Weather in {location}:")
    print(f"Temperature: {weather['temperature']}°{weather['unit'].capitalize()}")
    print(f"Wind Speed: {weather['wind_speed']} km/h")
    print(f"Weather Description: {weather['weather_description']}")


🌤️ Weather in Egypt:
Temperature: 33.8°Celsius
Wind Speed: 16.2 km/h
Weather Description: Clear sky ☀️


In [26]:
semantic_query = generate_semantic_query(user_input, weather)
print(f"🔎 Semantic Search Query: {semantic_query}")

🔎 Semantic Search Query: "outdoor clothing for hot weather in Egypt"


In [27]:
results = get_similarity_results(vector_db, semantic_query , k=3)

for i, doc in enumerate(results, 1):
    print(f"\n🔹 Match {i}:")
    print(doc.page_content)



🔹 Match 1:
these cool hours. Temperature Range: • High: 18°C to 24°C (i.e. 18°C, 19°C, 20°C, 21°C, 22°C, 23°C, 24°C) • Low: 10°C to 14°C (i.e. 10°C, 11°C, 12°C, 13°C, 14°C)  9. Hot and Dry Conditions Egypt Hot and dry conditions are a hallmark of Egypt’s desert climate, particularly during peak summer. These extreme conditions encourage early morning desert excursions, camel treks, and visits to historical sites before the heat intensifies. Ultra-lightweight, breathable clothing made of natural fibers,

🔹 Match 2:
layered thermal wear, beanies, gloves, and waterproof boots is essential. Temperature Range: • High: 2°C to 5°C (i.e. 2°C, 3°C, 4°C, 5°C) • Low: -1°C to 1°C (i.e. -1°C, 0°C, 1°C)  7. Windy Conditions Egypt Windy conditions are common in Egypt, particularly in desert areas and along the coast. The brisk winds create a refreshing counterbalance to the heat, making outdoor activities like kite flying, desert safaris, or coastal walks especially enjoyable. Lightweight fabrics, w

In [28]:
response = ask_llm_from_results(pipe, user_input , weather , results)
print(response)

Considering the current weather conditions in Egypt with a temperature of 33.8°Celsius and a clear sky, you should wear lightweight, breathable clothing made of natural fibers. Avoid dark colors which can absorb more heat. A hat or cap would also be beneficial to protect you from the sun. Given the hot and dry conditions, it's advisable to wear loose-fitting clothes that cover your skin to prevent sunburn. Don't forget to apply sunscreen to protect yourself from the sun's rays.


## User Interface

In [29]:
import gradio as gr

with gr.Blocks(title="🌦️ Smart Weather Assistant") as demo:
    gr.Markdown("## 🌦️ Smart Weather Assistant")
    gr.Markdown(
        "Ask about weather-based advice like clothing or activities. "
        "This assistant processes your query and returns location, weather, and advice."
    )

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(
                lines=3,
                label="Your Weather Query",
                placeholder="e.g. What should I wear in Luxor?"
            )
            submit_btn = gr.Button("Get Smart Advice")

        with gr.Column():
            location_out = gr.Textbox(label="📍 Extracted Location", interactive=False)
            weather_out = gr.Textbox(label="🌤️ Current Weather", interactive=False)
            answer_out = gr.Markdown(label="🧠 LLM Final Answer")

    submit_btn.click(
        fn=weather_assistant_verbose,
        inputs=user_input,
        outputs=[location_out, weather_out, answer_out]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://35de3b631be146265b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
